In [1]:
from pathlib import Path
import os

yr = 2023
d = 24

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  formatted_input = []
  for l in inp.splitlines():
    locs, velocities = l.split('@')
    formatted_input.append((tuple(int(x.strip()) for x in locs.split(',')),
     tuple(int(x.strip()) for x in velocities.split(','))))
  return formatted_input

In [3]:
import numpy as np




def find_intersection(l1, l2):
  '''
  Getting each hailstone into slope-intercept form
  then solving for the x location that they cross,
  followed by the y location
  '''


  def find_y_intercept(l):
    loc = l[0]
    vel = l[1]

    orig_x = loc[0]
    orig_y = loc[1]

    slope = np.divide(vel[1], vel[0])

    return np.add(loc[1], np.multiply(np.divide(vel[1], vel[0]), -loc[0]))

    
  def find_intersection_x(l1, l2):

    loc1 = l1[0]
    vel1 = l1[1]

    loc2 = l2[0]
    vel2 = l2[1]


    yi1 = find_y_intercept(l1)
    yi2 = find_y_intercept(l2)

    m1 = np.divide(vel1[1], vel1[0])
    m2 =  np.divide(vel2[1], vel2[0])


    # m1x + yi1 = m2x + yi2
    msum = np.subtract(m1, m2)
    ysum = np.subtract(yi2, yi1)


    x =  np.divide(ysum,msum)

    return x

  def find_intersection_y(l, x):
    loc = l[0]
    vel = l[1]

    return np.subtract(loc[1],
                       np.multiply(np.subtract(loc[0], x),
                                   np.divide(vel[1],vel[0]))
                       )



  x = find_intersection_x(l1, l2)
  y = find_intersection_y(l1, x)
  return (x,y)




def find_t(l, point):
  '''
  Given a hailstone and a point that we know is on the hailstone line,
  return the t at which the hailstone will reach the point
  '''
  loc = l[0]
  vel = l[1]

  dy = point[1] - loc[1]

  t = dy/vel[1]
  return t

In [4]:
def test_intersect(l1, l2, xrange=(200000000000000, 400000000000000), yrange=(200000000000000, 400000000000000)):
  '''
  Make sure the intersection is within the acceptable rectangle, and make sure it happens
  with a positive t for each hailstone
  '''
  intersection = find_intersection(l1, l2)
  t1, t2 = find_t(l1, intersection), find_t(l2, intersection)

  x = intersection[0]
  y = intersection[1]

  xs = [xrange[0], x, xrange[1]]
  ys = [yrange[0], y, yrange[1]]

  # Is the intersection forward or backward in time
  forward1 = t1 > 0
  forward2 = t2 > 0

  return xs == list(sorted(xs)) and ys == list(sorted(ys)) and forward1 and forward2, intersection, forward1, forward2


In [5]:

def is_parallel_2d(l1, l2):
  '''
  Checks for parallel lines by comparing
  slopes
  '''
  x1, y1 = l1[1][0], l1[1][1]
  x2, y2 = l2[1][0], l2[1][1]
  return x1/y1 == x2/y2


def count_intersections(formatted_input, xrange=(200000000000000, 400000000000000), yrange=(200000000000000, 400000000000000), debug=False):
  '''
  Loops over all the possible pairs of hailstones, and counts which ones satisfy our specifications
    - They are not parallel
    - They cross with a positive t (in the future) for both hailstones
    - The intersection point is within the given rectangle
  Returns the sum of satisifactory hailstone pairs
  '''
    
  from tqdm.notebook import tqdm

  count = 0
  if debug:
    r = range((len(formatted_input)))
  else:
    r = tqdm(range(len(formatted_input)))
  for i in r:
    for j in range(i+1, len(formatted_input)):
      assert(i!=j)
      hsa = formatted_input[i]
      hsb = formatted_input[j]
      parallel = is_parallel_2d(hsa, hsb)
      if not parallel:
        crossed, crossloc, forwardA, forwardB = test_intersect(hsa, hsb, xrange=xrange, yrange=yrange)
      else:
        crossed = False
      count += int(crossed)
      if debug:
        print('Hailstone A: {}, {}, {} @ {}, {}, {}'.format(hsa[0][0], hsa[0][1], hsa[0][2], hsa[1][0], hsa[1][1], hsa[1][2]))
        print('Hailstone B: {}, {}, {} @ {}, {}, {}'.format(hsb[0][0], hsb[0][1], hsb[0][2], hsb[1][0], hsb[1][1], hsb[1][2]))
        if parallel:
          print("Hailstones' paths are parallel; they never intersect.")
        elif not forwardA and not forwardB:
          print("Hailstones' paths crossed in the past for both hailstones.")
        elif not forwardA:
          print("Hailstones' paths crossed in the past for hailstone A.")
        elif not forwardB:
          print("Hailstones' paths crossed in the past for hailstone B.")
        elif not crossed:
          print("Hailstones' paths will cross outside the test area (at x={}, y={}).".format(round(crossloc[0], 4), round(crossloc[1], 4)))
        elif crossed:
          print("Hailstones' paths will cross inside the test area (at x={}, y={}).".format(round(crossloc[0], 4), round(crossloc[1], 4)))
        print('')
  return count

In [6]:
def find_throw(formatted_input):
    '''
    Just turning the first few hailstones into a system of
    equations and solving with Z3
    '''
    from z3 import Int, Solver
    
    # The rock
    xloc = Int('xloc')
    yloc = Int('yloc')
    zloc = Int('zloc')
    xvel = Int('xvel')
    yvel = Int('yvel')
    zvel = Int('zvel')

    t0 = Int('t0')
    t1 = Int('t1')
    t2 = Int('t2')
    
    s = Solver()
    s.add(
    formatted_input[0][0][0] + (formatted_input[0][1][0]*t0) == xloc + (xvel*t0),
    formatted_input[0][0][1] + (formatted_input[0][1][1]*t0) == yloc + (yvel*t0),
    formatted_input[0][0][2] + (formatted_input[0][1][2]*t0) == zloc + (zvel*t0),
    formatted_input[1][0][0] + (formatted_input[1][1][0]*t1) == xloc + (xvel*t1),
    formatted_input[1][0][1] + (formatted_input[1][1][1]*t1) == yloc + (yvel*t1),
    formatted_input[1][0][2] + (formatted_input[1][1][2]*t1) == zloc + (zvel*t1),
    formatted_input[2][0][0] + (formatted_input[2][1][0]*t2) == xloc + (xvel*t2),
    formatted_input[2][0][1] + (formatted_input[2][1][1]*t2) == yloc + (yvel*t2),
    formatted_input[2][0][2] + (formatted_input[2][1][2]*t2) == zloc + (zvel*t2)
    )
    s.check()

    
    return ((s.model()[xloc].as_long(), s.model()[yloc].as_long(), s.model()[zloc].as_long()), 
            (s.model()[xvel].as_long(), s.model()[yvel].as_long(), s.model()[zvel].as_long()))

def get_throw_sum(formatted_input):
    throw = find_throw(formatted_input)
    return np.sum(np.array(throw[0], dtype=np.longlong))

In [7]:
import time

t = time.time()

formatted_input = format_input(inp)

print(count_intersections(formatted_input, debug=False))
print(get_throw_sum(formatted_input))


print("\nRUNTIME: ", time.time()-t)

  0%|          | 0/300 [00:00<?, ?it/s]

15318
870379016024859

RUNTIME:  4.1539247035980225
